In [ ]:
from ImageNet import *

db_path = "/data/lab_ietr/hbrachem/ImageNet/"
dataset = ImageNet_dataset(os.path.join(db_path,"serialized_val.pickle"),db_path=os.path.join(db_path,"ILSVRC/Data/CLS-LOC/val/"))

In [ ]:
from model_utils import *

vlm = 'Llava-7b'
model,processor = vlm_pretrained(vlm)
device = model.device

In [ ]:
encoder = encoder_QKV(vlm,model)
w = processor.image_processor.crop_size['width']
h = processor.image_processor.crop_size['height']
patch_dim = 14

In [ ]:
num_targeted_layer = 2

attack_config = {
    'lr':"1e-3",
    'component': 'encoder',
    'layers':f"{num_targeted_layer}F",
    'bounding_box':'scaled*336',
    'algo':  "min_att",
    'loss':"att_mean",
    'optimizer':'adam',
    'steps':1000,
    'checkpoint':250,
}

target_layers_v = encoder["V"][:num_targeted_layer]
target_layers_k = encoder["K"][:num_targeted_layer]
target_layers_q = encoder["Q"][:num_targeted_layer]

In [ ]:
path = f"ablation_study/llava_{attack_config['component']}_{attack_config['layers']}/algo_{attack_config['algo']}/{attack_config['loss']}/{attack_config['optimizer']}_{attack_config['lr']}"

!mkdir -p $path
folder = path+'/adv_img'
!mkdir $folder
folder = path+'/hist'
!mkdir $folder
folder = path+'/predictions'
!mkdir $folder
folder = path+'/best'
!mkdir $folder

In [ ]:
from attack_utils import *

In [ ]:
import numpy as np

In [ ]:
for i ,[image,label,boxes,img_name] in enumerate(dataset):
    print(f"Processing sample number {i}...")
    if i == 50:
        break
        
    image = image.convert('RGB').resize((w,h))
    ref = np.array(image.convert('RGB')).copy()
    boxes = bounding_box_new_position(image,boxes,w,h)

    parametters = {'image':image}
    parametters['label'] = label
    parametters['boxes'] = boxes
    parametters['model'] = model
    parametters['processor'] = processor
    parametters['optimizer'] = attack_config["optimizer"]
    parametters['lr'] = float(attack_config['lr'])
    parametters['target_layers_q'] = target_layers_q
    parametters['target_layers_k'] = target_layers_k
    parametters['target_layers_v'] = target_layers_v
    parametters['lambda_a'] = 1
    parametters['lambda_e'] = 0
    parametters['lambda_n'] = 0
    parametters['w'] = w
    parametters['h'] = h
    parametters['patch_dim'] = patch_dim
    parametters['steps'] = attack_config["steps"]
    parametters['checkpoint'] = attack_config["checkpoint"]
    parametters['path'] = path
    parametters['img_name'] = img_name
    
    best_image, exec_time = generate_adv_image(**parametters)

    print(f"execution_time: {exec_time}")
    
    
    